<a href="https://colab.research.google.com/github/shrad1997/SVM/blob/main/SVM_Salary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SVM Classifier

In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
train_data=pd.read_csv("/content/SalaryData_Train(1).csv")
test_data=pd.read_csv("/content/SalaryData_Test(1).csv")

In [3]:
train_data.head(5)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [5]:
train_data['educationno'].unique()


array([13,  9,  7, 14,  5, 10, 12,  4, 16, 11, 15,  3,  6,  1,  8,  2])

In [6]:
from sklearn.preprocessing import LabelEncoder
LB=LabelEncoder()
train_data["workclass"]=LB.fit_transform(train_data["workclass"])
train_data["maritalstatus"]=LB.fit_transform(train_data["maritalstatus"])
train_data["occupation"]=LB.fit_transform(train_data["occupation"])
train_data["relationship"]=LB.fit_transform(train_data["relationship"])
train_data["race"]=LB.fit_transform(train_data["race"])
train_data["sex"]=LB.fit_transform(train_data["sex"])
train_data["native"]=LB.fit_transform(train_data["native"])
train_data["Salary"]=LB.fit_transform(train_data["Salary"])
train_data.head(5)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,5,Bachelors,13,4,0,1,4,1,2174,0,40,37,0
1,50,4,Bachelors,13,2,3,0,4,1,0,0,13,37,0
2,38,2,HS-grad,9,0,5,1,4,1,0,0,40,37,0
3,53,2,11th,7,2,5,0,2,1,0,0,40,37,0
4,28,2,Bachelors,13,2,9,5,2,0,0,0,40,4,0


In [7]:
train_data=train_data.drop(columns=['education'],axis=1)

In [8]:
train_data.head(5)

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,5,13,4,0,1,4,1,2174,0,40,37,0
1,50,4,13,2,3,0,4,1,0,0,13,37,0
2,38,2,9,0,5,1,4,1,0,0,40,37,0
3,53,2,7,2,5,0,2,1,0,0,40,37,0
4,28,2,13,2,9,5,2,0,0,0,40,4,0


In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   age            30161 non-null  int64
 1   workclass      30161 non-null  int64
 2   educationno    30161 non-null  int64
 3   maritalstatus  30161 non-null  int64
 4   occupation     30161 non-null  int64
 5   relationship   30161 non-null  int64
 6   race           30161 non-null  int64
 7   sex            30161 non-null  int64
 8   capitalgain    30161 non-null  int64
 9   capitalloss    30161 non-null  int64
 10  hoursperweek   30161 non-null  int64
 11  native         30161 non-null  int64
 12  Salary         30161 non-null  int64
dtypes: int64(13)
memory usage: 3.0 MB


In [10]:
train_data.describe()

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
count,30161.000000,30161.00000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000,30161.000000
mean,38.438115,2.19933,10.121316,2.580087,5.959849,1.418322,3.678592,0.675707,1092.044064,88.302311,40.931269,35.413481,0.248931
std,13.134830,0.95394,2.550037,1.498018,4.029633,1.601361,0.834721,0.468118,7406.466611,404.121321,11.980182,5.956430,0.432401
min,17.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,2.00000,9.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,37.000000,0.000000
50%,37.000000,2.00000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,37.000000,0.000000
75%,47.000000,2.00000,13.000000,4.000000,9.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,37.000000,0.000000
max,90.000000,6.00000,16.000000,6.000000,13.000000,5.000000,4.000000,1.000000,99999.000000,4356.000000,99.000000,39.000000,1.000000


In [11]:
#train_data['capitalloss'] = train_data['capitalloss'].astype(int)
#train_data['hoursperweek'] = train_data['hoursperweek'].astype(int)
train_data['capitalloss'] = train_data['capitalloss'].fillna(88.521935)
train_data['hoursperweek'] = train_data['hoursperweek'].fillna(41.078086)


In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_data[['age', 'workclass', 'educationno', 'maritalstatus', 'occupation',
       'relationship', 'race', 'sex', 'capitalgain', 'capitalloss',
       'hoursperweek', 'native']] = sc.fit_transform(train_data[['age', 'workclass', 'educationno', 'maritalstatus', 'occupation',
       'relationship', 'race', 'sex', 'capitalgain', 'capitalloss',
       'hoursperweek', 'native']])


In [13]:
train_data.head(5)

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,0.042779,2.935947,1.128898,0.947877,-1.479030,-0.261233,0.385055,0.692771,0.146085,-0.218508,-0.077735,0.266358,0
1,0.880261,1.887645,1.128898,-0.387243,-0.734533,-0.885712,0.385055,0.692771,-0.147447,-0.218508,-2.331495,0.266358,0
2,-0.033356,-0.208958,-0.439733,-1.722362,-0.238202,-0.261233,0.385055,0.692771,-0.147447,-0.218508,-0.077735,0.266358,0
3,1.108665,-0.208958,-1.224048,-0.387243,-0.238202,-0.885712,-2.010994,0.692771,-0.147447,-0.218508,-0.077735,0.266358,0
4,-0.794703,-0.208958,1.128898,-0.387243,0.754461,2.236683,-2.010994,-1.443479,-0.147447,-0.218508,-0.077735,-5.273965,0


In [14]:
train_data_x=train_data.iloc[:,0:12]
train_data_y=train_data.iloc[:,12]

In [15]:
test_data.head(5)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [16]:
from sklearn.preprocessing import LabelEncoder
LB=LabelEncoder()
test_data["workclass"]=LB.fit_transform(test_data["workclass"])
test_data["maritalstatus"]=LB.fit_transform(test_data["maritalstatus"])
test_data["occupation"]=LB.fit_transform(test_data["occupation"])
test_data["relationship"]=LB.fit_transform(test_data["relationship"])
test_data["race"]=LB.fit_transform(test_data["race"])
test_data["sex"]=LB.fit_transform(test_data["sex"])
test_data["native"]=LB.fit_transform(test_data["native"])
test_data["Salary"]=LB.fit_transform(test_data["Salary"])
test_data.head(5)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,2,11th,7,4,6,3,2,1,0,0,40,37,0
1,38,2,HS-grad,9,2,4,0,4,1,0,0,50,37,0
2,28,1,Assoc-acdm,12,2,10,0,4,1,0,0,40,37,1
3,44,2,Some-college,10,2,6,0,2,1,7688,0,40,37,1
4,34,2,10th,6,4,7,1,4,1,0,0,30,37,0


In [17]:
test_data=test_data.drop(columns=['education'],axis=1)

In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   age            15060 non-null  int64
 1   workclass      15060 non-null  int64
 2   educationno    15060 non-null  int64
 3   maritalstatus  15060 non-null  int64
 4   occupation     15060 non-null  int64
 5   relationship   15060 non-null  int64
 6   race           15060 non-null  int64
 7   sex            15060 non-null  int64
 8   capitalgain    15060 non-null  int64
 9   capitalloss    15060 non-null  int64
 10  hoursperweek   15060 non-null  int64
 11  native         15060 non-null  int64
 12  Salary         15060 non-null  int64
dtypes: int64(13)
memory usage: 1.5 MB


In [19]:
test_data.describe()

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
count,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,2.214874,10.112749,2.595286,5.989044,1.401394,3.683665,0.673772,1120.301594,89.041899,40.951594,35.467596,0.245684
std,13.380676,0.966422,2.558727,1.505336,4.020113,1.588954,0.827630,0.468848,7703.181842,406.283245,12.062831,5.880922,0.430506
min,17.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,2.000000,9.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,37.000000,0.000000
50%,37.000000,2.000000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,37.000000,0.000000
75%,48.000000,2.000000,13.000000,4.000000,9.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,37.000000,0.000000
max,90.000000,6.000000,16.000000,6.000000,13.000000,5.000000,4.000000,1.000000,99999.000000,3770.000000,99.000000,39.000000,1.000000


In [20]:
test_data['capitalloss'] = test_data['capitalloss'].fillna(89.041899)
test_data['hoursperweek'] = test_data['hoursperweek'].fillna(40.951594)

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
test_data[['age', 'workclass', 'educationno', 'maritalstatus', 'occupation',
       'relationship', 'race', 'sex', 'capitalgain', 'capitalloss',
       'hoursperweek', 'native']] = sc.fit_transform(test_data[['age', 'workclass', 'educationno', 'maritalstatus', 'occupation',
       'relationship', 'race', 'sex', 'capitalgain', 'capitalloss',
       'hoursperweek', 'native']])

In [22]:
test_data_x=test_data.iloc[:,0:12]
test_data_y=test_data.iloc[:,12]

In [23]:
test_data_x.head(5)

,age,workclass,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,-1.029005,-0.222347,-1.216563,0.933187,0.002725,1.006108,-2.034389,0.695832,-0.145438,-0.219169,-0.078889,0.260581
1,-0.057423,-0.222347,-0.434898,-0.395463,-0.494789,-0.881990,0.382230,0.695832,-0.145438,-0.219169,0.750131,0.260581
2,-0.804794,-1.257126,0.737599,-0.395463,0.997755,-0.881990,0.382230,0.695832,-0.145438,-0.219169,-0.078889,0.260581
3,0.391000,-0.222347,-0.044066,-0.395463,0.002725,-0.881990,-2.034389,0.695832,0.852624,-0.219169,-0.078889,0.260581
4,-0.356371,-0.222347,-1.607395,0.933187,0.251483,-0.252624,0.382230,0.695832,-0.145438,-0.219169,-0.907909,0.260581


In [24]:
test_data_y.head(5)



0    0
1    0
2    1
3    1
4    0
Name: Salary, dtype: int64

In [25]:
model_linear = SVC(kernel = "linear")
model_linear.fit(train_data_x,train_data_y)


SVC(kernel='linear')

In [26]:
y_pred = model_linear.predict(test_data_x)

In [27]:
#Generating the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_data_y, y_pred)

print(cm)

[[11012   348]
 [ 2526  1174]]


In [28]:
acc = accuracy_score(test_data_y, y_pred) * 100
print("accuracy= ",acc)

accuracy=  80.91633466135458


In [38]:
model2 = SVC(kernel = "rbf")
model2.fit(train_data_x,train_data_y)


SVC()

In [40]:
y_pred2 = model2.predict(test_data_x)

In [41]:
#Generating the confusion Matrix
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(test_data_y, y_pred2)

print(cm2)

[[10732   628]
 [ 1696  2004]]


In [42]:
acc2= accuracy_score(test_data_y, y_pred2) * 100
print("accuracy= ",acc2)

accuracy=  84.5683930942895
